In [1]:
import folium # library for maping
import json
import pandas as pd
import os
from branca.colormap import LinearColormap
from bokeh.io import export_png

In [2]:
# Resources
# Boundaries of the zipcodes
phx_geo = "resources/phoenix_zip_codes_geo_from_us_az_simplified_ms.json"
# file containing the values to plot
pv_file = os.path.join("resources","clean_pv_2018.csv")
# file to which we want the map to be saved
map_file = os.path.join("plots","zillow_pv_2018") #we will add the .html and .png extension
colorbar_caption = 'Zillow estimated median home value'
value_clabel = 'Zhvi'
zipcode_clabel = 'ZipCode'

In [1]:
# Loading the Property values
df = pd.read_csv(pv_file)
# df = df.loc[:, ["ZipCode", "Zhvi"]]
df[zipcode_clabel] = df[zipcode_clabel].astype(str)
# lets try to sort the values
df = df.dropna() # let's just try to remove the NaN rows
# Transforming the data into a dictionnary (to be used by folium.GeoJson)
# One Entry by zipcode and the correspondig Zhvi value
map_dict = df.set_index(zipcode_clabel)[zipcode_clabel].to_dict()

NameError: name 'pd' is not defined

In [4]:
color_scale = LinearColormap(['yellow','red'], vmin = min(map_dict.values()), vmax = max(map_dict.values()))

In [5]:
# defining the function that will look into the color_scale to pick the color corresponding to the value
def get_color(feature):
    value = map_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)

In [6]:
# Creating the map
m = folium.Map(location = [33.4484, -112.0740], zoom_start=8.5)
# Adding the colored zipcodes
folium.GeoJson(
    data = phx_geo,
    style_function = lambda feature: {
        'fillColor': get_color(feature),
        'fillOpacity': 0.7,
        'color' : 'black',
        'weight' : 1,
    }    
).add_to(m)
# Adding the colorbar
color_scale.caption = colorbar_caption
m.add_child(color_scale)

In [7]:
# Saving the map into an html file
m.save(outfile = map_file+".html")
# import subprocess
# outdir = "plots" # this directory has to exist..
# m.save("tmp.html")
# # url = "file://{}/tmp.html".format(os.getcwd())
# url = "file:///C:/Users/MariO/UA_data/PROJECT_1/tmp.html"
# print(url)
# outfn = os.path.join(outdir,"outfig.png")
# subprocess.check_call(["cutycapt","--url={}".format(url), "--out={}".format(outfn)])